# Use CMIP6 models together with observational data
This example demonstrates how to combine CMIP6 models with observational data from the Copernicus Climate Data Store (CDS). We have chosen Surface Downwelling Longwave Radiation (rlds variable) here. We will load the data, regrid the observational dataset and use it to validate the model. First, load all nesessary libraries.

In [ ]:
import intake
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xesmf as xe
import cf_xarray as cfxr
import ESMF

%matplotlib inline
print("Using xESMF in version %s" % xe.__version__)

# 1. Load the model
We then load one of the historical monthly CMIP6 models for December 2014 from Mistral, look at its metadata, and plot it.

In [ ]:
col = intake.open_esm_datastore("/pool/data/Catalogs/dkrz_cmip6_disk.json")
cat = col.search(variable_id = "rlds",
                 table_id = "Amon",
                 activity_id = "CMIP",
                 experiment_id = "historical",
                source_id = "MPI-ESM1-2-HR",
                member_id = "r1i1p1f1")
xr_dict = cat.to_dataset_dict()
dset = xr_dict[list(xr_dict.keys())[0]]
dset['rlds']

In [ ]:
fig = plt.figure(figsize = (25, 10))
ax = fig.add_subplot(1, 1, 1, projection = ccrs.Mollweide())

plt.contourf(dset['rlds'].sel(time = '2014-12-16T12:00:00')[0].lon,
            dset['rlds'].sel(time = '2014-12-16T12:00:00')[0].lat,
            dset['rlds'].sel(time = '2014-12-16T12:00:00')[0], 60,
            transform = ccrs.PlateCarree(0),
            cmap = 'coolwarm')
ax.coastlines()
ax.set_global()

# Add a color bar
plt.colorbar(ax = ax)
plt.title('Surface Downwelling Longwave Radiation in Dec 2014 (CMIP6), W m-2', 
          fontdict = {'fontsize' : '24', 'fontweight' : 'bold'} )
plt.show()

## 2. Load the observational data
As an observational dataset we chose the Surface Downwelling Longwave Radiation derived from AHVRR satellite. We download it from the Climate Data Store.
https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-surface-radiation-budget?tab=form

### In sections 2.0 - 2.1 we describe how to retrive the observational dataset from CDS. For the purpose of this workshop, however, we have already prepared this dataset, so you can directly load it in section 2.2.

### 2.0 CDS API
In order to do so we used CDS API which we need to set up first. Please refer to the source git repository for the details on the installation and use of this package.
https://github.com/ecmwf/cdsapi

You would need to enter your UID and your API Key to use CDS.

In [ ]:
#!pip install cdsapi
# cat > ~/.cdsapirc
# url: https://cds.climate.copernicus.eu/api/v2
# key: <your UID>:<your API Key>
# verify: 0
#pip install cfgrib

In [ ]:
# import cdsapi
# import cfgrib
# cds = cdsapi.Client()

### 2.1 Downlod the data from the CDS
We download the dataset for the same month as our modeling daset, unzip it, load it to the notebook, check its metadata, and finally plot it.

In [ ]:
# cds.retrieve(
#     'satellite-surface-radiation-budget',
#     {
#         'format': 'tgz',
#         'product_family': 'clara',
#         'origin': 'eumetsat',
#         'variable': 'surface_downwelling_longwave_flux',
#         'time_aggregation': 'monthly_mean',
#         'climate_data_record_type': 'thematic_climate_data_record',
#         'year': '2014',
#         'month': '12',
#         'version': 'v2_0',
#         'sensor_on_satellite': 'avhrr_on_multiple_satellites',
#     },
#     'download.tar.gz')

In [ ]:
#!tar -xvf download.tar.gz SDLmm20141201000000219AVPOS01GL.nc -C /obs/

In [ ]:
obs = xr.open_dataset("../../data/SDLmm20141201000000219AVPOS01GL.nc").isel(time = 0)
obs

In [ ]:
fig = plt.figure(figsize = (25, 10))
ax = fig.add_subplot(1, 1, 1, projection = ccrs.Mollweide())

plt.contourf(obs['SDL'].lon, obs['SDL'].lat, obs['SDL'], 60,
            transform = ccrs.PlateCarree(),
            cmap = 'coolwarm')
ax.coastlines()
ax.set_global()

# Add a color bar
plt.colorbar(ax = ax)
plt.title('Surface Downwelling Longwave Radiation in Dec 2014 (AVHRR), W m-2', 
          fontdict = {'fontsize' : '24', 'fontweight' : 'bold'} )
plt.show()

## 3. Regridding
As we could find out from the metadata of both datasets, their dimesions are different. We therefore need to regrid one of the datasets before we could spatially overlay them. We will use xESMF and ESMF libraries for this purpose. Please refer to the respective documentation to see the details and parameters of the functions from these packages (https://xesmf.readthedocs.io/en/latest/index.html, https://github.com/esmf-org/esmf).

In [ ]:
# In case of problems, activate ESMF verbose mode
ESMF.Manager(debug = True)

# Regridding methods
method_list = ['bilinear','nearest_s2d', 'conservative', 'conservative_normed', 'patch']

# Function to generate the weights
# If grids have problems of degenerated cells near the poles there is the ignore_degenerate option
def regrid(ds_in, ds_out, method, periodic, unmapped_to_nan = True, ignore_degenerate = None):
    """Convenience function for calculating regridding weights"""
    return xe.Regridder(ds_in, ds_out, method, periodic = periodic, 
                        unmapped_to_nan = unmapped_to_nan, 
                        ignore_degenerate = ignore_degenerate)

Create a regridder for converting the observational dataset to match the model grid.

In [ ]:
regridder = regrid(obs, dset, "bilinear", periodic = True, unmapped_to_nan = True, ignore_degenerate = None) 
regridder

In [ ]:
#Regrid the observational dataset and write write it to the xarray with the modeling dataset.
dset["AVHRR"] = regridder(obs.SDL, keep_attrs = True, skipna = True, na_thres = 1.0)
dset

## 4. Validate the model
Finally, we can create a difference map between our datasets and calculate its corresponding bias and RMSE.

In [ ]:
# Calculate the differene map
diff = dset['AVHRR'] - dset['rlds'].sel(time = '2014-12-16T12:00:00')
diff = diff[:,:,'member_id' == 'r1i1p1f1']

#Calculate the bias and the RMSE 
bias = diff.mean().values
rmse = np.sqrt(np.square(diff).sum()/np.square(diff).size).values

# Plot the results
fig = plt.figure(figsize = (25, 10))
ax = fig.add_subplot(1, 1, 1, projection = ccrs.Mollweide())

plt.contourf(diff.lon, diff.lat, diff,
            transform = ccrs.PlateCarree(),
            cmap = 'coolwarm')
ax.coastlines()
ax.set_global()

# Print the bias and the RMSE in a textbox
textstr = '\n'.join((
    r'bias = %.2f' % (bias, ),
    r'RMSE = %.2f' % (rmse, )))

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)
ax.text(0.05, 0.95, textstr, transform = ax.transAxes, fontsize = 17,
        verticalalignment = 'top', bbox = props)

# Add a color bar and a title
plt.colorbar(ax = ax)
plt.title('Difference map between CMIP6 and AVHRR-derived Surface Downwelling Longwave Radiation, W m-2', 
          fontdict = {'fontsize' : '24', 'fontweight' : 'bold'} )
plt.show()